In [3]:
import os

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [3]:
! pwd

DATA_PATH = Path("kion_train")
DATA_PATH

/home/iuliiasolomennikova/!!!RecSysService/notebooks


PosixPath('kion_train')

In [4]:
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

In [5]:
Columns.Datetime = 'last_watch_dt'


# примеведем все к Datetime
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

# if user wathed more than 10, 3, 1, percent - set 3 балла этому взаим-ию
# if less - 1 балл
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)


In [6]:
# отправим в test  max  interaction for the last week
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [7]:
# drop записи где длина less 300
train.drop(train.query("total_dur < 300").index, inplace=True)



In [8]:
# отфильтруем cold пользователей from test
cold_users = set(test[Columns.User]) - set(train[Columns.User])

# in test only users with info about interaction
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)


### User features

In [9]:
users.isnull().sum()


user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [10]:
users.fillna('Unknown', inplace=True)
users.nunique()


user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [11]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()


In [12]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head(10)
# конструируем фичи

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex
8,846063,Ж,sex
9,401219,Ж,sex
11,312520,Ж,sex
12,555088,Ж,sex
13,382508,М,sex


### Item features

In [13]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [14]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items.nunique()


item_id         14019
content_type        2
title           13454
title_orig       9724
release_year      104
genres           2559
countries         666
for_kids            2
age_rating          6
studios            38
directors        7414
actors          11830
description     13791
keywords        13583
dtype: int64

### Genre

In [15]:
# make the table

items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head(4)

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre


In [16]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

content_feature.head(3)

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type


In [17]:
item_features = pd.concat((genre_feature, content_feature))
item_features.head(3)

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre


### Метрики:

In [20]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

## Модель:

 LightFM - лучше чем SVD and ALS 

 A hybrid latent representation recommender model - model learns embeddings 
   

In [21]:
K_RECOS = 10      # num of recomendations
RAND_ST = 777
THREADS = 16
N_FACTORS = (32, 64, 128, 160)      # num of factors
ITERATIONS = (15, 20)

best_components = 17 # dimensionality of the feature latent embeddings.
best_loss = 'logistic'
best_rho = 0.93 # moving average coefficient for the adadelta learning schedule.
best_lr = 0.05

epsilon = 3 # onditioning parameter for the adadelta learning schedule.

dataset = Dataset.construct(
        interactions_df=train)

lightfm = LightFMWrapperModel(
        model = LightFM(
            no_components = best_components,
            learning_schedule = 'adadelta',
            loss = best_loss,
            rho = best_rho,
            epsilon = epsilon,
            learning_rate = best_lr,
            random_state = RAND_ST
        )
    )
lightfm.fit(dataset)

recs = lightfm.recommend(
    users=test[Columns.User].unique(),
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)

metric_values = calc_metrics(metrics, recs, test, train)
model = lightfm

In [24]:
# ! pip install nmslib 
# 

In [25]:
import nmslib
#  методом приближенного поиска соседей для выдачи рекомендаций

import time


**Non-Metric Space Library** (*NMSLIB*) is an efficient cross-platform similarity search library and a toolkit for evaluation of similarity search methods. The core-library does not have any third-party dependencies.

Goal - searching in generic and non-metric spaces.


**parameters**:

- `ef` - the size of the dynamic list for the nearest neighbors (used during the search) - регулирует как быстро и точно ищем вектора. Сколько соседей перебираем
- `k` - number of nearest neighbors 
- `M` - the number of bi-directional links created for every new element during construction - также регулирует скорость
- `ef_construction` - controls the index_time/index_accuracy
- `num_elements` - defines the maximum number of elements in the index






In [26]:
user_embeddings, item_embeddings = model.get_vectors(dataset)
# vestors for users and items

user_embeddings.shape, item_embeddings.shape


((756562, 19), (14019, 19))

In [27]:
#add additional dimensions
def aug_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [28]:
max_norm, augmented_item_embeddings = aug_inner_product(item_embeddings)
augmented_item_embeddings.shape

print('initial item shape: ', item_embeddings.shape)
print('augmented item shape: ', augmented_item_embeddings.shape)



initial item shape:  (14019, 19)
augmented item shape:  (14019, 20)


In [30]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))  # add dimension for users
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape
print('augmented users shape: ', augmented_item_embeddings.shape)




augmented users shape:  (14019, 20)


In [31]:
user_id = 30

In [32]:
print(user_embeddings[user_id])
print(augmented_user_embeddings[user_id])  # аугментация для users

[ 1.44839287e-04  1.00000000e+00 -3.95889161e-03 -1.83553342e-02
 -2.04228368e-02 -1.30793601e-02 -4.57607862e-03 -2.14211736e-02
 -2.50692014e-02  3.99297453e-04 -1.04909772e-02 -1.45006296e-03
  1.75321307e-02  4.33024950e-03 -2.50951890e-02 -2.82166395e-02
  1.71820237e-03 -1.08103342e-02  1.51334703e-02]
[ 1.44839287e-04  1.00000000e+00 -3.95889161e-03 -1.83553342e-02
 -2.04228368e-02 -1.30793601e-02 -4.57607862e-03 -2.14211736e-02
 -2.50692014e-02  3.99297453e-04 -1.04909772e-02 -1.45006296e-03
  1.75321307e-02  4.33024950e-03 -2.50951890e-02 -2.82166395e-02
  1.71820237e-03 -1.08103342e-02  1.51334703e-02  0.00000000e+00]


In [33]:
item_id = 0

print(item_embeddings[item_id])
print(augmented_item_embeddings[item_id] # аугментация для items
)

[ 1.00000000e+00  3.03673887e+00 -3.26367617e-02  2.59880275e-02
 -1.98296513e-02 -1.75734311e-02 -1.61644034e-02 -1.82163797e-03
  2.34975629e-02  3.66074182e-02 -1.34112062e-02  8.00592825e-03
 -4.32435572e-02 -3.09442542e-02 -2.11636350e-02 -5.08000366e-02
 -7.43114110e-03 -1.09344655e-04  2.51392350e-02]
[ 1.00000000e+00  3.03673887e+00 -3.26367617e-02  2.59880275e-02
 -1.98296513e-02 -1.75734311e-02 -1.61644034e-02 -1.82163797e-03
  2.34975629e-02  3.66074182e-02 -1.34112062e-02  8.00592825e-03
 -4.32435572e-02 -3.09442542e-02 -2.11636350e-02 -5.08000366e-02
 -7.43114110e-03 -1.09344655e-04  2.51392350e-02  1.26507354e+01]


### Set index parameters


In [35]:
M = 64  # how fast we are searching
efc = 128   # how long index will be build
threads = 4

# Number of neighbors 
K=10

# max inner product
space_name='negdotprod' # что мы ищем



index_time_params = {'M': M, 'indexThreadQty': threads, 'efConstruction': efc, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 64, 'indexThreadQty': 4, 'efConstruction': 128, 'post': 0}


In [36]:
# start library, specify  space,  type of vector and add data points
# слои, гравф


index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

14019

In [37]:
# Create  index

start = time.time()
index_time_params = {'M': M, 'indexThreadQty': threads, 'efConstruction': efc}

index.createIndex(index_time_params) 
end = time.time() 

print('Index-time parameters', index_time_params)
print('Index time = %f' % (end-start))

Index-time parameters {'M': 64, 'indexThreadQty': 4, 'efConstruction': 128}
Index time = 1.140332


In [38]:

# Setting query-time parameters
efs = 128
query_time_params = {'efSearch': efs}

print('Setting query-time parameters', query_time_params)

index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 128}


In [39]:
# Querying

query_matrix = augmented_user_embeddings

query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = threads)

end = time.time() 
print('kNN time overall=%f (seconds), per query=%f (sec), per query adjusted for number threads=%f (sec)' % 
      (end-start, float(end-start)/query_qty, threads*float(end-start)/query_qty))

kNN time overall=8.118877 (seconds), per query=0.000011 (sec), per query adjusted for number threads=0.000043 (sec)


In [40]:
# take id with top score

def recom_all(query_factors, index_factors, topn=10):
    output = query_factors.dot(index_factors.T)
    argpart_indices = np.argpartition(output, -topn)[:, -topn:]

    x_indices = np.repeat(np.arange(output.shape[0]), topn)
    y_indices = argpart_indices.flatten()
    top_value = output[x_indices, y_indices].reshape(output.shape[0], topn)
    top_indices = np.argsort(top_value)[:, ::-1]

    y_indices = top_indices.flatten()
    top_indices = argpart_indices[x_indices, y_indices]
    labels = top_indices.reshape(-1, topn)
    
    distance = output[x_indices, top_indices].reshape(-1, topn)
    return labels, distance

In [41]:
recom_all(user_embeddings[[0], :], item_embeddings)


(array([[12982,  8867, 11758,  8287,  3527,  3174,  2256,   118,  5797,
          7342]]),
 array([[13.00890718, 12.89064559, 12.49925029, 12.4245779 , 12.18878199,
         12.04518037, 11.80307593, 11.45031093, 11.33914716, 11.17126055]]))

In [42]:
query_matrix_not_augmented = user_embeddings[:1000, :]


In [43]:
recom_all(query_matrix_not_augmented, item_embeddings)


(array([[12982,  8867, 11758, ...,   118,  5797,  7342],
        [12982,  8867, 11758, ...,   118,  5797,  7342],
        [12982,  8867, 11758, ...,   118,  5797,  7342],
        ...,
        [12982,  8867, 11758, ...,   118,  5797,  7342],
        [12982,  8867, 11758, ...,   118,  5797,  7342],
        [12982,  8867, 11758, ...,   118,  5797,  7342]]),
 array([[13.00890718, 12.89064559, 12.49925029, ..., 11.45031093,
         11.33914716, 11.17126055],
        [13.0099134 , 12.89014008, 12.49320571, ..., 11.45273143,
         11.34035989, 11.16867941],
        [13.37757462, 13.25915064, 12.8652308 , ..., 11.81537676,
         11.70073991, 11.53538346],
        ...,
        [14.80288003, 14.69002252, 14.28634937, ..., 13.24185435,
         13.13031566, 12.95876639],
        [13.01518825, 12.89615087, 12.50640051, ..., 11.45954414,
         11.34150095, 11.17826681],
        [13.02748611, 12.91361029, 12.5222728 , ..., 11.46759599,
         11.35453932, 11.18895594]]))

In [44]:
index.knnQueryBatch(query_matrix, k = K, num_threads = threads)


[(array([12982,  8867, 11758,  8287,  3527,  3174,  2256,   118,  5797,
          7342], dtype=int32),
  array([-13.008907, -12.890647, -12.499249, -12.424577, -12.188782,
         -12.045181, -11.803076, -11.45031 , -11.339148, -11.171262],
        dtype=float32)),
 (array([12982,  8867, 11758,  8287,  3527,  3174,  2256,   118,  5797,
          7342], dtype=int32),
  array([-13.0099125, -12.89014  , -12.493206 , -12.426633 , -12.188279 ,
         -12.037206 , -11.804351 , -11.452731 , -11.34036  , -11.168679 ],
        dtype=float32)),
 (array([12982,  8867, 11758,  8287,  3527,  3174,  2256,   118,  5797,
          7342], dtype=int32),
  array([-13.377576 , -13.2591505, -12.865231 , -12.795172 , -12.552093 ,
         -12.406023 , -12.17208  , -11.815377 , -11.70074  , -11.535384 ],
        dtype=float32)),
 (array([12982,  8867, 11758,  8287,  3527,  3174,  2256,   118,  5797,
          7342], dtype=int32),
  array([-13.044341 , -12.926089 , -12.536088 , -12.471366 , -12.21739  ,
  

In [47]:
# ! pip install hnswlib

import hnswlib


In [48]:
max_elements, dim = augmented_item_embeddings.shape
hnsw = hnswlib.Index("ip", dim) # might be L2, cosine or ip (innder product)

# Init index - the max number of elems must be known beforehand
hnsw.init_index(max_elements, M, efc)

# Element insertion (can be called several times)
hnsw.add_items(augmented_item_embeddings)

In [49]:
# Control recall by setting efc, where efc > k
hnsw.set_ef(efc)

label, distance = hnsw.knn_query(query_matrix, k=k)


In [51]:
labels, distances = recom_all(user_embeddings[:1000, :], item_embeddings)

print(labels)
print()
print(distances)

[[12982  8867 11758 ...   118  5797  7342]
 [12982  8867 11758 ...   118  5797  7342]
 [12982  8867 11758 ...   118  5797  7342]
 ...
 [12982  8867 11758 ...   118  5797  7342]
 [12982  8867 11758 ...   118  5797  7342]
 [12982  8867 11758 ...   118  5797  7342]]

[[13.00890718 12.89064559 12.49925029 ... 11.45031093 11.33914716
  11.17126055]
 [13.0099134  12.89014008 12.49320571 ... 11.45273143 11.34035989
  11.16867941]
 [13.37757462 13.25915064 12.8652308  ... 11.81537676 11.70073991
  11.53538346]
 ...
 [14.80288003 14.69002252 14.28634937 ... 13.24185435 13.13031566
  12.95876639]
 [13.01518825 12.89615087 12.50640051 ... 11.45954414 11.34150095
  11.17826681]
 [13.02748611 12.91361029 12.5222728  ... 11.46759599 11.35453932
  11.18895594]]


In [53]:
# Save item_embeddings
# save user_embeddings



import joblib


                 
                 
joblib.dump(item_embeddings, '/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/ANN_item_embeddings.sav')
joblib.dump(user_embeddings, '/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/ANN_user_embeddings.sav')


joblib.dump(label, '/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/ANN_label.sav')
joblib.dump(distance, '/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/ANN_distance.sav')

['/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/ANN_distance.sav']